<a href="https://colab.research.google.com/github/JayP127/cfp/blob/main/cfpSearch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This Google Colab notebook presents an LLM approach to the accessibility of carbon footprint information of products and services, a pressing need in our increasingly environmentally-conscious society. This investigation contributes to sustainability and transparency, empowering consumers and companies with vital knowledge to make informed decisions that minimize environmental impact.

The programming language of this notebook is Python, a versatile and widely-accepted language known for its simplicity and robustness. Leveraging the capabilities of Python, we've harnessed the power of the Langchain library, a tool that has been instrumental in handling language processing tasks.

Our approach uses an Agent that employs three different tools for information retrieval.

**Pinecone vector store:** This tool stores and manages vector embeddings of data that represents the carbon footprint of products and services. The embeddings are derived from several comprehensive datasets, including GWP100 Global warming potential average of CML v4.8 2006, EDIP 2003, IPCC2021, ReCiPe 2016 v1.3, and a carbon catalog of 866 products from an article published in Nature. The dataset consists of 22,105 vectors (products), each with 1536 dimensions. Pinecone's efficient similarity search capabilities help us expediently find the most relevant information. When the model cites pinecone as the source of information, it refers to the dataset contained in this Pinecone Vectorstore.

**Google Search Tool:** A vital and expansive source of information, the Google Search Tool serves as our gateway to the vast information available on the internet. The Search tool allows our Agent to broaden its search and cover more ground.

**Calculator:** Our Agent incorporates a calculator tool to perform necessary computations and produce estimates when needed.

Our Agent is designed to deliver the best possible answer to queries about product carbon footprints, striving to cite sources wherever possible. The amalgamation of these techniques provides a picture of the carbon footprint information landscape.

In this notebook, we take you through the entire process: from how we've integrated these different tools to form an effective agent to how we've leveraged this Agent to uncover and evaluate the carbon footprint data.

LLM applications always have room for improvement as sometimes will lead to hallucinations or answers that are not real. However, it is clear to us that research in the field of LLMs is moving fast; with the advent of more capable LLMs, larger and more accurate datasets, better tools, and prompt writing, these improvements will be significant in the near future.

To work with the model below, you need to obtain four API keys from the following websites:


OPENAI_API_KEY: https://openai.com/ \\
PINECONE_API_KEY: https://www.pinecone.io/ \\
PINECONE_ENV: https://www.pinecone.io/ \\
SERPAPI_API_KEY: https://serpapi.com/ \\



In [1]:
!pip -q install openai python-dotenv langchain pinecone-client tiktoken pinecone-text google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 29.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [4]:
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
import textwrap

In [5]:
embeddings = OpenAIEmbeddings()

In [6]:
import pinecone
# initialize pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENV'],
)

index_name = "cfp"

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Uploading the vectorstore- only needed for the initialization

In [ ]:
loader = CSVLoader('/content/drive/MyDrive/CFP_CSVs/binder3.csv')

In [ ]:
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [ ]:
products=text_splitter.split_documents(documents)

In [ ]:
print(products[0])
print(products[1])
print(products[2])

page_content='\ufeff: 0\nGeography: USA\nproduct_name: Frosted Flakes, 23 oz, produced in Lancaster, PA (one carton)---Kellogg Company\nreference_product_unit: unit\nCarbon Footprint of Product (kg CO2-Eq): 2.67\nSource of information: Meinrenken, Christoph J., Daniel Chen, Ricardo A. Esparza, Venkat Iyer, Sally P. Paridis, Aruna Prasad, and Erika Whillas. "The Carbon Catalogue, carbon footprints of 866 commercial products from 8 industry sectors and 5 continents."Â\xa0Scientific DataÂ\xa09, no. 1 (2022): 87.' metadata={'source': '/content/drive/MyDrive/CFP_CSVs/binder3.csv', 'row': 0}
page_content='\ufeff: 1\nGeography: USA\nproduct_name: Office Chair---KNOLL INC\nreference_product_unit: unit\nCarbon Footprint of Product (kg CO2-Eq): 3.51\nSource of information: Meinrenken, Christoph J., Daniel Chen, Ricardo A. Esparza, Venkat Iyer, Sally P. Paridis, Aruna Prasad, and Erika Whillas. "The Carbon Catalogue, carbon footprints of 866 commercial products from 8 industry sectors and 5 conti

Initial uploading to the vector store

In [ ]:
docsearch = Pinecone.from_documents(products, embeddings, index_name=index_name)

Example of an initial basic query

In [ ]:
query= "what is the carbon footprint of jeans"
products = docsearch.similarity_search(query)

In [ ]:
print(products[0].page_content)

﻿: 10
Geography: USA
product_name: 501Â® Original Jeans â€“ Rinse Run---Levi Strauss & Co.
reference_product_unit: unit
Carbon Footprint of Product (kg CO2-Eq): 15.05
Source of information: Meinrenken, Christoph J., Daniel Chen, Ricardo A. Esparza, Venkat Iyer, Sally P. Paridis, Aruna Prasad, and Erika Whillas. "The Carbon Catalogue, carbon footprints of 866 commercial products from 8 industry sectors and 5 continents."Â Scientific DataÂ 9, no. 1 (2022): 87.


# Once the index is created in the Vector Store, you can create an access to it

In [7]:
llm = ChatOpenAI(temperature=0, model="gpt-4")

In [8]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)

climate_change = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

# Create the Agent

In [9]:
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

In [10]:
tools = [
    Tool(
        name="CFP_pinecone",
        func=climate_change.run,
        description="Useful to answer questions on the carbon footprint of products, this tool should be used first. Always cite source. Input should be in the form of a question containing full context."
    ),
    Tool(
        name="Search",
        func=search.run,
        description="Useful to answer questions on the carbon footprint of products, this tool should be used second. Always cite source. Input should be in the form of a question containing full context."
    ),
    Tool.from_function(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

In [11]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [12]:
question = "What is the average carbon fooprint of oat milk?. Cite Sources"
output = agent.run(question)

# Ensure output is a string, then wrap it
if isinstance(output, str):
    wrapped_output = textwrap.fill(output, width=80)
else:
    # Assuming the output is a non-string (like a list or dict), we convert it to a string first
    output = str(output)
    wrapped_output = textwrap.fill(output, width=80)

print(wrapped_output)



> Entering new  chain...

Invoking: `CFP_pinecone` with `What is the average carbon footprint of oat milk?`


I'm sorry, but the information provided does not include the carbon footprint of oat milk.
Invoking: `Search` with `What is the average carbon footprint of oat milk?`


Oat Milk Carbon Footprint: Carbon Footprint of Oat Milk Many find it creamier, and eco-conscious users love that it has one of the lowest overall footprints. The carbon emissions are at 0.18 kg for every 200 ml.The average carbon footprint of oat milk is 0.18 kg per 200 ml. This means that oat milk has one of the lowest overall carbon footprints among dairy alternatives, making it a popular choice among eco-conscious consumers. [Source](https://www.bbc.co.uk/news/science-environment-46654042)

> Finished chain.
The average carbon footprint of oat milk is 0.18 kg per 200 ml. This means that
oat milk has one of the lowest overall carbon footprints among dairy
alternatives, making it a popular choice among eco-co

In [13]:
question = "What is the carbon fooprint of cow's milk?. Cite Sources"
output = agent.run(question)

# Ensure output is a string, then wrap it
if isinstance(output, str):
    wrapped_output = textwrap.fill(output, width=80)
else:
    # Assuming the output is a non-string (like a list or dict), we convert it to a string first
    output = str(output)
    wrapped_output = textwrap.fill(output, width=80)

print(wrapped_output)



> Entering new  chain...

Invoking: `CFP_pinecone` with `What is the carbon footprint of cow's milk?`


The carbon footprint of cow's milk varies depending on the geographical location. Here are some examples:

- Globally, the carbon footprint is 1.597103092 kg CO2-Eq per kg of cow milk.
- In Canada- Quebec, the carbon footprint is 1.290145411 kg CO2-Eq per kg of cow milk.
- In South Africa, the carbon footprint is 1.126659567 kg CO2-Eq per kg of cow milk.
- In the rest of the world, the carbon footprint is 1.453046046 kg CO2-Eq per kg of cow milk.The carbon footprint of cow's milk varies depending on the geographical location. Here are some examples:

- Globally, the carbon footprint is 1.597103092 kg CO2-Eq per kg of cow milk.
- In Canada- Quebec, the carbon footprint is 1.290145411 kg CO2-Eq per kg of cow milk.
- In South Africa, the carbon footprint is 1.126659567 kg CO2-Eq per kg of cow milk.
- In the rest of the world, the carbon footprint is 1.453046046 kg CO2-Eq per kg of cow

In [14]:
question = "What is the average carbon fooprint of electricity from photovoltaics?. Cite Sources"
output = agent.run(question)

# Ensure output is a string, then wrap it
if isinstance(output, str):
    wrapped_output = textwrap.fill(output, width=80)
else:
    # Assuming the output is a non-string (like a list or dict), we convert it to a string first
    output = str(output)
    wrapped_output = textwrap.fill(output, width=80)

print(wrapped_output)



> Entering new  chain...

Invoking: `CFP_pinecone` with `What is the average carbon footprint of electricity from photovoltaics?`


The information provided does not give the average carbon footprint of electricity from photovoltaics. It only provides the carbon footprint for specific photovoltaic products and installations.
Invoking: `Search` with `What is the average carbon footprint of electricity from photovoltaics?`


There have been many studies on the carbon footprint of solar panels with varying results. The Intergovernmental Panel on Climate Change (IPCC) found the median value among peer-reviewed studies for life-cycle emissions for rooftop solar is 41 grams of CO2 equivalent per kilowatt hour of electricity produced.The average carbon footprint of electricity from photovoltaics, according to the Intergovernmental Panel on Climate Change (IPCC), is approximately 41 grams of CO2 equivalent per kilowatt hour of electricity produced. This value is a median derived from various

In [15]:
question = "What is the average carbon fooprint of electricity in the United States?. Cite Sources"
output = agent.run(question)

# Ensure output is a string, then wrap it
if isinstance(output, str):
    wrapped_output = textwrap.fill(output, width=80)
else:
    # Assuming the output is a non-string (like a list or dict), we convert it to a string first
    output = str(output)
    wrapped_output = textwrap.fill(output, width=80)

print(wrapped_output)



> Entering new  chain...

Invoking: `CFP_pinecone` with `What is the average carbon footprint of electricity in the United States?`


The information provided does not give a clear average carbon footprint of electricity in the United States. The carbon footprint for medium voltage electricity varies from 0.152984903 kg CO2-Eq to 0.472636304 kg CO2-Eq per kWh, and for high voltage electricity it is 0.46641302 kg CO2-Eq per kWh. However, one of the entries does not provide a carbon footprint value. Therefore, an exact average cannot be calculated from the given data.The carbon footprint of electricity in the United States varies depending on the voltage level. For medium voltage electricity, the carbon footprint ranges from 0.152984903 kg CO2-Eq to 0.472636304 kg CO2-Eq per kWh. For high voltage electricity, the carbon footprint is 0.46641302 kg CO2-Eq per kWh. Please note that these values are averages and actual emissions can vary based on factors such as the mix of energy sources u

In [17]:
question = "What is the average carbon fooprint of electricity from wind turbines?. Give examples and cite Sources"
output = agent.run(question)

# Ensure output is a string, then wrap it
if isinstance(output, str):
    wrapped_output = textwrap.fill(output, width=80)
else:
    # Assuming the output is a non-string (like a list or dict), we convert it to a string first
    output = str(output)
    wrapped_output = textwrap.fill(output, width=80)

print(wrapped_output)



> Entering new  chain...

Invoking: `CFP_pinecone` with `What is the average carbon footprint of electricity from wind turbines?`


The information provided does not specify the average carbon footprint of electricity from wind turbines. It only provides the carbon footprint of specific wind turbine units of different power capacities.
Invoking: `Search` with `What is the average carbon footprint of electricity from wind turbines?`


99% lower emissions than fossil fuels When you divide the total emissions associated with a wind turbine by the amount of electricity it will produce in its lifetime, it works out at about 6 g of carbon dioxide for every kilowatt-hour (kWh) of electricity.The average carbon footprint of electricity from wind turbines is approximately 6 grams of carbon dioxide for every kilowatt-hour (kWh) of electricity. This is significantly lower than the emissions from fossil fuels, with wind turbines producing 99% lower emissions. 

Please note that the carbon footpr

In [19]:
question = "What is the average carbon fooprint of beef?. Cite Sources"
output = agent.run(question)

# Ensure output is a string, then wrap it
if isinstance(output, str):
    wrapped_output = textwrap.fill(output, width=80)
else:
    # Assuming the output is a non-string (like a list or dict), we convert it to a string first
    output = str(output)
    wrapped_output = textwrap.fill(output, width=80)

print(wrapped_output)



> Entering new  chain...

Invoking: `CFP_pinecone` with `What is the average carbon footprint of beef?`


The information provided does not specify the average carbon footprint of beef. It only provides the carbon footprint for "red meat, live weight" which could include other types of meat besides beef.
Invoking: `Search` with `What is the average carbon footprint of beef?`


What Is the Carbon Footprint of Meat and What Is Beef Carbon Footprint Per Kg? The carbon footprint of beef is 27 kg per kilogram and is one of the highest levels of other animal and plant-based foods.The average carbon footprint of beef is 27 kg of CO2 equivalent per kilogram. This is one of the highest levels among other animal and plant-based foods. [Source](https://www.carbonfootprint.com/food_carbon_footprints.html)

> Finished chain.
The average carbon footprint of beef is 27 kg of CO2 equivalent per kilogram.
This is one of the highest levels among other animal and plant-based foods.
[Source](https://www

In [20]:
question = "What is the average carbon fooprint of vegetables?. Cite Sources"
output = agent.run(question)

# Ensure output is a string, then wrap it
if isinstance(output, str):
    wrapped_output = textwrap.fill(output, width=80)
else:
    # Assuming the output is a non-string (like a list or dict), we convert it to a string first
    output = str(output)
    wrapped_output = textwrap.fill(output, width=80)

print(wrapped_output)



> Entering new  chain...

Invoking: `CFP_pinecone` with `What is the average carbon footprint of vegetables?`


The information provided does not give the average carbon footprint of all vegetables. It only provides the carbon footprint for specific products: lettuce and cauliflower. The carbon footprint for lettuce is given as 3.686253246 kg CO2-Eq/kg and 0.166278457 kg CO2-Eq/kg in two different sources. The carbon footprint for cauliflower is given as 0.328882909 kg CO2-Eq/kg.
Invoking: `Search` with `What is the average carbon footprint of vegetables?`


Most vegetables are produced with less than 2.00 kg CO2 emission per kilogram of food.The carbon footprint of vegetables can vary greatly depending on the specific type of vegetable. For instance, the carbon footprint for lettuce is given as 3.686253246 kg CO2-Eq/kg and 0.166278457 kg CO2-Eq/kg in two different sources, and the carbon footprint for cauliflower is given as 0.328882909 kg CO2-Eq/kg [source: CFP Pinecone]. However, 

In [22]:
question = "What is the average CO2 emissions per mile for an ICE automobile?. Cite Sources"
output = agent.run(question)

# Ensure output is a string, then wrap it
if isinstance(output, str):
    wrapped_output = textwrap.fill(output, width=80)
else:
    # Assuming the output is a non-string (like a list or dict), we convert it to a string first
    output = str(output)
    wrapped_output = textwrap.fill(output, width=80)

print(wrapped_output)



> Entering new  chain...

Invoking: `CFP_pinecone` with `What is the average CO2 emissions per mile for an ICE (Internal Combustion Engine) automobile?`


The information provided does not specify the average CO2 emissions per mile for an Internal Combustion Engine (ICE) automobile.
Invoking: `Search` with `What is the average CO2 emissions per mile for an ICE (Internal Combustion Engine) automobile?`


A typical passenger vehicle emits about 4.6 metric tons of carbon dioxide per year. This number can vary based on a vehicle's fuel, fuel economy, and the number of miles driven per year.A typical passenger vehicle, which is usually an Internal Combustion Engine (ICE) automobile, emits about 4.6 metric tons of carbon dioxide per year. However, this number can vary based on factors such as the type of fuel used, the fuel economy of the vehicle, and the number of miles driven per year [source](https://www.epa.gov/greenvehicles/greenhouse-gas-emissions-typical-passenger-vehicle).

> Finis

In [23]:
question = "What is the average carbon footprint of a bicycle?. Cite Sources"
output = agent.run(question)

# Ensure output is a string, then wrap it
if isinstance(output, str):
    wrapped_output = textwrap.fill(output, width=80)
else:
    # Assuming the output is a non-string (like a list or dict), we convert it to a string first
    output = str(output)
    wrapped_output = textwrap.fill(output, width=80)

print(wrapped_output)



> Entering new  chain...

Invoking: `CFP_pinecone` with `What is the average carbon footprint of a bicycle?`


The average carbon footprint of a bicycle varies based on geography. Globally, it is 148.7098401 kg CO2-Eq per unit. In the rest of the world, it is 150.4141163 kg CO2-Eq per unit. In Europe, it is 145.2588964 kg CO2-Eq per unit.The average carbon footprint of a bicycle varies based on geography. Globally, it is approximately 148.71 kg CO2-Eq per unit. In the rest of the world, it is approximately 150.41 kg CO2-Eq per unit. In Europe, it is approximately 145.26 kg CO2-Eq per unit. These figures take into account the entire lifecycle of the bicycle, including production, use, and disposal. [Source: Pinecone Database](https://www.pinecone.com)

> Finished chain.
The average carbon footprint of a bicycle varies based on geography. Globally,
it is approximately 148.71 kg CO2-Eq per unit. In the rest of the world, it is
approximately 150.41 kg CO2-Eq per unit. In Europe, it is ap